# Lecture 1: Breaking RSA with SPA - Analyze

In [1]:
import sys
sys.path.insert(0, '..')
import securec
import securec.util as util
scope, target = util.init()

## Code to attack

### Implementation

Let us recap the code from the previous notebook:
```c
uint8_t exponent = private_exponent;
uint8_t message = 0xA0;

uint16_t tmp;
uint8_t result = 1;
while (exponent)
{
    if (exponent & 1)
    {
        tmp = result * message;
        result = tmp % modulus;
    }

    tmp = message * message;
    message = tmp % modulus;
    exponent >>= 1;
}
```

Our expectation is that we can see a difference between a round executing the "multiply" step and a round executing the "multiply and square" step. As the difference only depends on the exponent let's modify the program to feed the exponent directly.

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 1</div>

1. Have a look at `2_rsa_uint8_variable.c`. What is different to `1_rsa_uint8_fixed`. Are all differences clear?
2. Insert a cell and check if the program works correctly. Therefore, use the functions `target.simpleserial_write` and `target.simpleserial_read`.

</div>

### Record multiple traces and recognize differences

In [2]:
securec.util.compile_and_flash('./2_rsa_uint8_variable.c')

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2077 bytes
✓


In [3]:
import struct
import time
import warnings
    
scope.default_setup()
scope.adc.samples = 5500

def capture(exponent, message=0xA0, modulus=0xFF):
    scope.arm()
    target.simpleserial_write(0x01, struct.pack('<BBB', message, exponent, modulus))
    return util.capture()

In [4]:
traces = [(exp, capture(exp)) for exp in (0b100, 0b101, 0b111)]

In [5]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool
from bokeh.palettes import Category10_10

output_notebook()

Loading BokehJS ...

In [6]:
p = figure(width=900, height=800)
p.add_tools(CrosshairTool())
for i, (exp, trace) in enumerate(traces):
    p.line(range(0, len(trace)), trace + i * 0.6, legend_label=bin(exp), line_color=Category10_10[i])
show(p)

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 2</div>

Explain the plot above!

</div>

<div style="background: #f0ffe0; padding: 15px; border: 1px solid slategray;">
<div class="h2" style="font-variant: small-caps;">Exercise 3</div>

Now you have an idea how different exponents look like. Go back to the previous notebook [1. SPA RSA Introduction](1.%20SPA%20RSA%20Introduction.ipynb). Modify the C-code in [1_rsa_uint8_fixed.c](1_rsa_uint8_fixed.c) and modify the exponent there. Flash the program and exchange the chipwhisperer with another student. Reveal her or his exponent!

</div>

In [7]:
util.exit()